In [7]:
import os 
from dotenv import load_dotenv
load_dotenv()
groqapi_key=os.getenv("GROQ_API_KEY")



In [8]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it",api_key=groqapi_key)

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage,HumanMessage



In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store={}

def get_session_history(session_id:str )->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(model,get_session_history=get_session_history)


In [11]:
config={"configurable":{"session_id":"chat1"}}


In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi I ma ashish aspiring full stack developper witl llm knowledge")],
    config=config
)


In [ ]:
response.content


In [ ]:
with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config
)

In [ ]:
## cahnge the config
config={"configurable":{"session_id":"chat1"}}
with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config
)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [

        ("system","You are a helpful assistant. Answer all the question to the rest of your ability"),
        MessagesPlaceholder(variable_name="messages")
        
    ]
)

chain=prompt|model
chain.invoke({"messages":[HumanMessage(content="Hi my name is B ashish")]})



In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)


In [ ]:
config={"configurable":{"session_id":"chat3"}}
with_message_history.invoke(
    [HumanMessage(content=" what is my anme")],
    config=config   
)

In [44]:
prompts=ChatPromptTemplate.from_messages(
    [
        ("system","You are an AI Assistant answer everything in {language}Langauge only"),
        MessagesPlaceholder(variable_name="messages")
        
    ]
)

chain=prompts|model


In [45]:
withmessage_history=RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="messages"
)

In [46]:
config={"configurable":{"session_id":"chat4"}}

In [ ]:
response=withmessage_history.invoke({"messages":[HumanMessage(content=" Hi I am a sigma male")],"language":"Hindi"},
                                    config=config)
response.content

In [ ]:
from langchain_core.messages import SystemMessage,trim_messages,AIMessage
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)



In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(

    RunnablePassthrough.assign(messages=itemgetter("messages")| trimmer)
    |prompt
    |model
)
response=chain.invoke(
  {
    "messages":messages+ [HumanMessage(content="what ice cream do i like")],
    
     "language":"English"}),
   

response

In [ ]:
response=chain.invoke(
  {
    "messages":messages+ [HumanMessage(content="what math proble did i ask ")],
    
     "language":"English"}),
   

response

In [ ]:
messagehistort=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"

)
config={"configurable":{"session_id":"chat5"}}

messagehistort.invoke(
    {
        "messages":messages+ [HumanMessage(content="What is amth problem asked")]
    }
)